In [ ]:
# idha bech ta3mel training w tjib el dataset men roboflow t7ot hedhom w tal9a el dataset 3andek
#hedhi t7otha bech el collab ya3mel install bel s7i7 ll dataset
import locale
locale.getpreferredencoding = lambda: "UTF-8"
# na3melou install ll dataset mte3na men roboflow
!pip install roboflow
from roboflow import Roboflow
rf = Roboflow(api_key="ho1o9PhgzNHGeAfQ6Uw9")
project = rf.workspace("roboflow-100").project("vehicles-q0x2v")
version = project.version(2)
dataset = version.download("yolov9")


In [ ]:
# el train bech ysir 3al collab b 7okem GPU a9wa w mba3ed ne5dhou fichier el weights njarboha 3ana
pip install ultralytics
from ultralytics import YOLO
# na5tarou model el yolo eli bech ne5dmou w bih w ahawka tal9ahom el modelet fi site ultralytics
model = YOLO('yolov9m.pt')



# na3mlou el train w lena 7atit 100 epochs w 6 tsawer fi kol batch w na3tih path el fichier yamel (eli feha el classes wel path mt3 train w test w val (idha 3andek))
results = model.train(data='/content/vehicles-2/data.yaml',epochs=100,batch=6)

In [3]:
#hedha el kol ta3mlou fel pc mte3ek 5ater fel collab mee tnajamech t7el el camera
# import ll bib eli na7tejouhom
import cv2
import numpy as np
from ultralytics import YOLO
import os
import pandas as pd
os.chdir(r"C:\Users\toumi\OneDrive\Bureau\rr") # nbadlou el dir eli ne5dmou fih win mawjouda el dataset mte3na wel fichier best.pt
model=YOLO('best.pt') # hedha el fichier eli 3malnelou download mel collab weli fih el weights
cap=cv2.VideoCapture("4K Video of Highway Traffic!.mp4") # na3mlou test 3ala vd idha t7eb 3ala camera el pc 7ot blassek kel path 0
classNames=["car","truck"] # hedhom el classes mte3na
# el arearight wel arealeft tala3nehom 3an tri9 5dhina cap ll video w mchina ll google ktebna robflow polygone zone w rsamna el ROI w houa ya3tina el coordonnees
arearight=[np.array([[327, 122], [379, 126], [640, 219], [332, 330]])] # hedhi el era eli bech na3mlou feha el count (3al imin)
arealeft=[np.array([[312, 123], [254, 125], [3, 208], [312, 329]])] # hedhi el era eli bech na3mlou feha el count (3al isar)

while(1):
    f,frame=cap.read() # na9raw el retun mt3 el camera wela taswira n5abioha fel frame wel f mee na7tejouhech fel model heki t9olna fama frames wela lee
    if f is None :
        break
    # tnajem ta3mel resize ll frame ama rod belek lee t7ot a9al mel size el aslenjiya 5ater el boxes bech yjiou b3ad exp : new_frame=cv2.resize(frame,(500,500))
    result=model(frame) # na3mlou el prediction mte3na
    
    
    a=result[0].boxes.data # ne5dhou el resultat eli feha el classses wel x1y1x2y2
    a=a.cpu().numpy() # nrodou el resultat men tensors ll numpy bech enajmou n7otoha 3al CPU 5ater mee 3anech GPU
    pc=pd.DataFrame(a) # na3mloha dataframe bech nsahlou el access mte3na ll data
    # hedhom zouz listet fer8in taw ta3ref dawrhom trayef o5ra
    rightnbr=[]
    leftnbr=[]
    #na3mlou loop 3ala kel dataframe eli 3ana w ne5dhou el classes wel boxes
    for index , row in pc.iterrows():
        # nrodouhom el kol int 5ater 7ajetna bihom int values
        x1=int(row[0]) 
        y1=int(row[1])
        x2=int(row[2])
        y2=int(row[3])  
        pred=int(row[5])
        xcenter=int(x1+x2)//2 # hedha center el bounding boxes taw ta3ref ech 7ajetna bih trayef o5ra
        ycenter=int(y1+y2)//2      
        rightzone=cv2.pointPolygonTest(np.array(arearight,np.int32), ((xcenter,ycenter)),measureDist=False) # hedhi function fel cv2 wadhifetha ta3malek kima ROI bech t9olou rahou mee yhemni ken fi kel region heki 
        leftzone=cv2.pointPolygonTest(np.array(arealeft,np.int32), ((xcenter,ycenter)),measureDist=False)
        cv2.polylines(frame,arearight,True,(0, 255, 0), 2) # norsmou kel region mte3na eli na7na 7ajetna beha w 5dhineha mel pointPolygonTest
        cv2.polylines(frame,arealeft,True,(0, 0, 255), 2)

        # hedhom el arg eli bech n3adiouhom fi fct putText
        org = [x1, y1] # mnin bech nabdew nektou
        font = cv2.FONT_HERSHEY_SIMPLEX # style el ktiba
        fontScale = 1 # size el ktiba parrapport el image
        color = (255, 0, 0) # el couleur (B,G,R) el couleur eli t7eb 3lih 7ot fih 255
        
        thickness = 1 # 8oledh el ktiba
        #cc=cv2.pointPolygonTest(np.array(area,np.int32), ((xcenter,ycenter)),measureDist=False)

        #print(aa)
        #print(bb)
        if (rightzone>=0) : # hedha ma3neha idha fama karhba 3al imin el pointPolygonTest return mte3ha bech ykoun 0 wela 1 idha lee -1
            cv2.rectangle(frame,(x1, y1), (x2, y2), (0, 0, 255), thickness) # norsmou el rectangle mte3na (bounding boxes)

            
            cv2.putText(frame, classNames[pred], org, font, fontScale, color, thickness) # nektbou ech n7ebou 
            rightnbr.append([xcenter]) # na3mlou append ll centre mt3 el boundig boxes ll list

            
        # taw nafess la7keya ama 3ala el chira el issar     
        elif (leftzone>=0) :
            
        
            cv2.rectangle(frame,(x1, y1), (x2, y2), (0, 0, 255), thickness)
            leftnbr.append([ycenter])
            
            cv2.putText(frame, classNames[pred], org, font, fontScale, color, thickness)
        #print(len(rightnbr))
        #print(len(leftnbr))
    
    rnbr= len(rightnbr) # ne7ssbou toul el list mte3na bech na3rfou 3dad el kraheb 

    lnbr= len(leftnbr)
    
    cv2.putText(frame, "nbr of cars in right way is =" + str(rnbr), (100,320), font, fontScale, color, 2)
    cv2.putText(frame, "nbr of cars in left way is =" + str(lnbr), (7,20), font, fontScale, color, 2)


    cv2.imshow('High way', frame) # na3mlou show ll frame 
    if cv2.waitKey(1) == ord('q'): # nenzlou 3al q bech nsakrou el window 
       break

cap.release()
cv2.destroyAllWindows() # nsakrou el windows el mahloulin el kol   


0: 384x640 18 cars, 1 truck, 653.4ms
Speed: 5.1ms preprocess, 653.4ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 14 cars, 2 trucks, 619.5ms
Speed: 2.5ms preprocess, 619.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 13 cars, 1 truck, 616.1ms
Speed: 2.1ms preprocess, 616.1ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 14 cars, 1 truck, 711.3ms
Speed: 2.0ms preprocess, 711.3ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 14 cars, 1 truck, 861.7ms
Speed: 3.0ms preprocess, 861.7ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 15 cars, 1 truck, 855.6ms
Speed: 3.4ms preprocess, 855.6ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 15 cars, 1 truck, 835.6ms
Speed: 3.3ms preprocess, 835.6ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 17 cars, 1 truck, 837.5ms
Speed: 3.